In [1]:
!pip install scikit-learn fastapi uvicorn pyngrok nest_asyncio joblib pandas

In [2]:
# create folder and write files
import os
os.makedirs('/content/smartspend', exist_ok=True)
%cd /content/smartspend


/content/smartspend


In [3]:
%%bash
python - <<'PY'
import pandas as pd, numpy as np, joblib
from sklearn.ensemble import IsolationForest
from datetime import datetime, timedelta

# create synthetic dataset
np.random.seed(42)
rows=[]
start = datetime.now() - timedelta(days=90)
for i in range(5000):
    uid = f"user_{i%200}"
    t = start + timedelta(minutes=30*i)
    amount = abs(np.random.normal(loc=50+(i%10), scale=20))
    cat = np.random.choice(['food','transport','shopping','bills','entertainment'])
    rows.append([f"tx_{i}", uid, t.isoformat(), amount, cat, "merchant_x"])
df = pd.DataFrame(rows, columns=['transaction_id','user_id','timestamp','amount','category','merchant'])
df.to_csv('transactions.csv', index=False)

# basic features for training (use amount + hour)
df['hour'] = pd.to_datetime(df['timestamp']).dt.hour
X = df[['amount','hour']].copy()
X['amount_log'] = np.log1p(X['amount'])
X = X[['amount_log','hour']]

# train IsolationForest
model = IsolationForest(n_estimators=100, contamination=0.01, random_state=42)
model.fit(X)
joblib.dump(model, 'model.joblib')
print("Created transactions.csv and trained model.joblib")
PY


Created transactions.csv and trained model.joblib


In [15]:
%%writefile api_server.py
from fastapi import FastAPI
from pydantic import BaseModel
import joblib, pandas as pd, uuid, os, numpy as np
from datetime import datetime

app = FastAPI(title="SmartSpend AI Financial API")

# -------------------------
# File paths
# -------------------------
MODEL_ANOMALY_PATH = "models/model_anomaly.joblib"
MODEL_CATEGORY_PATH = "models/model_category.joblib"
MODEL_FORECAST_PATH = "models/model_forecast.joblib"
TX_CSV = "transactions.csv"

# -------------------------
# Load models
# -------------------------
model_anomaly = joblib.load(MODEL_ANOMALY_PATH) if os.path.exists(MODEL_ANOMALY_PATH) else None
model_category = joblib.load(MODEL_CATEGORY_PATH) if os.path.exists(MODEL_CATEGORY_PATH) else None
# Forecast model can be user-specific or trained dynamically

# In-memory transaction storage
transactions = []

# -------------------------
# Pydantic Models
# -------------------------
class Transaction(BaseModel):
    user_id: str
    timestamp: str   # ISO format
    amount: float
    category: str = None
    merchant: str = None

class Question(BaseModel):
    user_id: str
    question: str

# -------------------------
# Helper functions
# -------------------------
def make_features(df):
    df = df.copy()
    df['hour'] = pd.to_datetime(df['timestamp']).dt.hour
    df['amount_log'] = df['amount'].apply(lambda x: np.log1p(x) if x>=0 else 0)
    return df[['amount_log','hour']]

# -------------------------
# Endpoints
# -------------------------

@app.get("/health")
def health():
    return {
        "status": "ok",
        "model_anomaly_loaded": model_anomaly is not None,
        "model_category_loaded": model_category is not None,
        "transactions_file_exists": os.path.exists(TX_CSV)
    }

@app.post("/transactions")
def add_transaction(tx: Transaction):
    tx_id = str(uuid.uuid4())
    row = {
        "transaction_id": tx_id,
        "user_id": tx.user_id,
        "timestamp": tx.timestamp,
        "amount": tx.amount,
        "category": tx.category or "",
        "merchant": tx.merchant or ""
    }
    transactions.append(row)
    # Save to CSV for persistence
    df = pd.DataFrame([row])
    if not os.path.exists(TX_CSV):
        df.to_csv(TX_CSV, index=False)
    else:
        df.to_csv(TX_CSV, mode='a', header=False, index=False)
    return {"transaction_id": tx_id, "message": "Transaction added"}

@app.get("/transactions")
def list_transactions():
    return {"transactions": transactions}

@app.post("/predict_anomaly")
def predict_anomaly(tx: Transaction):
    if model_anomaly is None:
        return {"error":"Anomaly model not loaded"}
    df = pd.DataFrame([tx.dict()])
    feat = make_features(df)
    score = model_anomaly.decision_function(feat)[0]
    threshold = -0.2
    is_anomaly = score < threshold
    return {"anomaly_score": float(score), "is_anomaly": bool(is_anomaly)}

@app.post("/predict_category")
def predict_category(tx: Transaction):
    if model_category is None:
        return {"error":"Category model not loaded"}
    df = pd.DataFrame([tx.dict()])
    # Example features: amount, hour
    feat = make_features(df)
    pred_cat = model_category.predict(feat)[0]
    return {"predicted_category": pred_cat}

@app.get("/budget/{user_id}")
def recommend_budget(user_id: str):
    if not os.path.exists(TX_CSV):
        return {"error":"No transactions yet"}
    df = pd.read_csv(TX_CSV, parse_dates=['timestamp'])
    user_df = df[df['user_id']==user_id]
    if user_df.empty:
        return {"user_id": user_id, "message":"No transactions yet"}
    out = {}
    for cat, g in user_df.groupby('category'):
        monthly_avg = g['amount'].sum() / 3.0
        recommended = monthly_avg * 1.1
        out[cat] = {"monthly_avg": round(monthly_avg,2), "recommended_budget": round(recommended,2)}
    return {"user_id": user_id, "budgets": out}

@app.get("/summary/{user_id}")
def transaction_summary(user_id: str):
    user_tx = [t for t in transactions if t['user_id']==user_id]
    if not user_tx:
        return {"user_id": user_id, "message":"No transactions yet"}
    df = pd.DataFrame(user_tx)
    summary = {
        "total_transactions": len(df),
        "total_amount": df['amount'].sum(),
        "average_amount": df['amount'].mean(),
        "max_transaction": df['amount'].max(),
        "min_transaction": df['amount'].min()
    }
    return {"user_id": user_id, "summary": summary}

@app.get("/category-spending/{user_id}")
def category_spending(user_id: str):
    user_tx = [t for t in transactions if t['user_id']==user_id]
    if not user_tx:
        return {"user_id": user_id, "message":"No transactions yet"}
    df = pd.DataFrame(user_tx)
    category_sum = df.groupby('category')['amount'].sum().to_dict()
    return {"user_id": user_id, "category_spending": category_sum}

@app.get("/predict_forecast/{user_id}")
def predict_forecast(user_id: str):
    # Optional: simple naive forecast using last 3 months average
    if not os.path.exists(TX_CSV):
        return {"error":"No transactions yet"}
    df = pd.read_csv(TX_CSV, parse_dates=['timestamp'])
    user_df = df[df['user_id']==user_id]
    if user_df.empty:
        return {"user_id": user_id, "message":"No transactions yet"}
    daily_avg = user_df['amount'].resample('D', on='timestamp').sum().rolling(30).mean().iloc[-1]
    forecast = daily_avg * 30  # next month approx
    return {"user_id": user_id, "forecast_next_month": round(float(forecast),2)}

@app.post("/ask_finance")
def ask_finance(q: Question):
    # Mock LLM response for demo (replace with real LLM API like OpenAI GPT)
    user_tx = [t for t in transactions if t['user_id']==q.user_id]
    total_spent = sum([t['amount'] for t in user_tx])
    response = f"User {q.user_id} has spent approx ${total_spent:.2f}. Question asked: '{q.question}'. Advice: Keep tracking your categories and reduce overspending."
    return {"answer": response}


Overwriting api_server.py


In [16]:
# --------------------------
# Start FastAPI server + ngrok (adjusted)
# --------------------------
from pyngrok import ngrok
import nest_asyncio, os, time

nest_asyncio.apply()  # allow nested event loops in Colab

# Prompt for ngrok authtoken
NGROK_TOKEN = input("Enter your ngrok authtoken: ").strip()
if not NGROK_TOKEN:
    raise SystemExit("Ngrok authtoken is required to start the server.")

# Authenticate ngrok
!ngrok authtoken {NGROK_TOKEN}

# Kill any existing tunnels
ngrok.kill()

# Start uvicorn FastAPI server in background
cmd = "nohup python -m uvicorn api_server:app --host 0.0.0.0 --port 8000 &"
os.system(cmd)
print("Starting FastAPI server in background...")
time.sleep(2)  # give it a moment to start

# Open ngrok tunnel on port 8000
public_url = ngrok.connect(8000)
print("Public URL:", public_url)
print("Swagger UI available at:", f"{public_url}/docs")

Enter your ngrok authtoken: 321P0jjQBijXzA67xjrB0y78YGC_2EFx9k9PDdP8NA3dt6PLv
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Starting FastAPI server in background...
Public URL: NgrokTunnel: "https://4b26d3986118.ngrok-free.app" -> "http://localhost:8000"
Swagger UI available at: NgrokTunnel: "https://4b26d3986118.ngrok-free.app" -> "http://localhost:8000"/docs


In [17]:
import requests, json
base = "https://4b26d3986118.ngrok-free.app"
# example health
print(requests.get(f"{base}/health").json())

# add transaction
tx = {"user_id":"user_1", "timestamp":"2025-08-30T12:00:00", "amount":250.0, "category":"shopping", "merchant":"storeA"}
print("add:", requests.post(f"{base}/transactions", json=tx).json())

# predict
print("predict:", requests.post(f"{base}/predict", json=tx).json())

# budget
print("budget:", requests.get(f"{base}/budget/user_1").json())


{'status': 'ok', 'model_loaded': True}
add: {'transaction_id': '335120ed-cf53-42c5-93ca-dc230ca0f270'}
predict: {'anomaly_score': 0.0962358224329587, 'is_anomaly': False}
budget: {'user_id': 'user_1', 'budgets': {'bills': {'monthly_avg': 68.84, 'recommended_budget': 75.72}, 'entertainment': {'monthly_avg': 67.62, 'recommended_budget': 74.38}, 'food': {'monthly_avg': 113.44, 'recommended_budget': 124.79}, 'shopping': {'monthly_avg': 377.25, 'recommended_budget': 414.97}, 'transport': {'monthly_avg': 67.01, 'recommended_budget': 73.71}}}
